In [447]:
import pandas as pd
import numpy as np
import tensorflow as tf
import psycopg2
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [448]:
# Informações de conexão
db_params = {
  'user': 'postgres',
  'password': 'postgres',
  'host': 'localhost',
  'dbname': 'GOALS',
}

# Conecta ao PostgreSQL
conn = psycopg2.connect(**db_params)

# Cria um cursor para executar comandos SQL
cursor = conn.cursor()

In [449]:
# Query Historico
query = """SELECT * FROM futebol.match_history"""
historico = pd.read_sql_query(query, conn)

C:\Users\salvv\AppData\Local\Temp\ipykernel_8392\1784265378.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  historico = pd.read_sql_query(query, conn)


In [450]:
historico.head()

,game_id,date,comp,round,venue,result,gf,ga,opponent,xg,...,fls,off,recov,season,team,days,time_diff,home_team_id,away_team_id,league_id
0,0,2023-04-15,Série A,1,Home,W,2,1,sao_paulo,0.9,...,20,2,57,2023,botafogo_rj,208,0.812207,0,8,0
1,1,2023-04-24,Série A,2,Away,W,2,1,bahia,0.2,...,9,5,54,2023,botafogo_rj,199,0.819550,0,14,0
2,2,2023-04-30,Série A,3,Away,W,3,2,flamengo,1.9,...,12,1,42,2023,botafogo_rj,193,0.824482,0,4,0
3,3,2023-05-07,Série A,4,Home,W,2,0,atl_mineiro,2.0,...,14,0,43,2023,botafogo_rj,186,0.830274,0,5,0
4,4,2023-05-11,Série A,5,Home,W,3,0,corinthians,2.6,...,12,0,54,2023,botafogo_rj,182,0.833601,0,12,0


In [451]:
data = historico.drop(['game_id', 'date', 'comp', 'round', 'season', 'league_id', 'gf', 'ga', 'opponent', 'team'], axis= 1)

In [452]:
#data = historico[['venue', 'result', 'xg', 'home_team_id', 'away_team_id', 'gf', 'ga']]

In [453]:
data.columns

Index(['venue', 'result', 'xg', 'xga', 'poss', 'sh', 'sot', 'saves', 'cmp',
       'att', 'prgp', 'kp', 'pass_3rd', 'sw', 'crs', 'sca', 'gca', 'tkl',
       'tklw', 'tkl_def_3rd', 'tkl_att_3rd', 'blocks', 'int',
       'touches_att_3rd', 'fls', 'off', 'recov', 'days', 'time_diff',
       'home_team_id', 'away_team_id'],
      dtype='object')

In [454]:
data['venue'] = pd.factorize(data['venue'])[0]
data['result'] = pd.factorize(data['result'])[0]

In [455]:
y = data.loc[:, 'result']
X = data.drop('result', axis= 1)

In [456]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=67)

In [457]:
X.shape

(1508, 30)

In [458]:
inputs = tf.keras.Input(shape=(30,))
x = tf.keras.layers.Dense(32, activation='relu')(inputs)
x = tf.keras.layers.Dense(32, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


batch_size = 32
epochs = 20

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau()]
)

Epoch 1/20


27/27 [==============================] - 1s 13ms/step - loss: 9.9143 - accuracy: 0.3626 - auc: 0.4637 - val_loss: 1.8407 - val_accuracy: 0.4171 - val_auc: 0.4860 - lr: 0.0010
Epoch 2/20
27/27 [==============================] - 0s 5ms/step - loss: 1.2973 - accuracy: 0.3874 - auc: 0.5041 - val_loss: 0.4787 - val_accuracy: 0.4502 - val_auc: 0.6243 - lr: 0.0010
Epoch 3/20
27/27 [==============================] - 0s 5ms/step - loss: 0.6487 - accuracy: 0.4182 - auc: 0.5802 - val_loss: 0.6384 - val_accuracy: 0.4739 - val_auc: 0.6586 - lr: 0.0010
Epoch 4/20
27/27 [==============================] - 0s 3ms/step - loss: 0.5252 - accuracy: 0.4289 - auc: 0.6214 - val_loss: 0.2412 - val_accuracy: 0.4502 - val_auc: 0.6880 - lr: 0.0010
Epoch 5/20
27/27 [==============================] - 0s 3ms/step - loss: 0.1878 - accuracy: 0.4360 - auc: 0.6737 - val_loss: 0.1576 - val_accuracy: 0.4882 - val_auc: 0.7223 - lr: 0.0010
Epoch 6/20
27/27 [==============================] - 0s 3ms/step - loss: 0.2702 - accu

In [459]:
model.evaluate(X_test, y_test)

15/15 [==============================] - 0s 3ms/step - loss: -0.9333 - accuracy: 0.4724 - auc: 0.7798


[-0.9333097338676453, 0.47240617871284485, 0.7797674536705017]